# Aggregating counts and rates of hospital admissions and make a table

In [10]:
import pandas as pd
import numpy as np
import functools as ft

## 0- functions and read data

In [11]:
def data_process(data, data_hosped, infection):
    #replace SMs with 5 in data and data_hosped and NaN with 0 in data_hosped 
    data = data.replace({'SM': 5})
    data_hosped = data_hosped.replace({'SM': 5, np.nan:0})
    #conver counts into int
    data[infection] = data[infection].astype(str).replace('\.0', '', regex=True).astype(int)
    data_hosped[infection] = data_hosped[infection].astype(str).replace('\.0', '', regex=True).astype(int)

    #add hosp rate
    data_hosped[infection+'_rate'] = (data_hosped[infection]/data[infection])*1000
    data_hosped[infection+'_rate'] = np.round(data_hosped[infection+'_rate'], decimals=1)

    #replace rate with SN if infection count equals 5
    data_hosped.loc[data_hosped[infection] == 5, infection+'_rate'] = 'SN'

    #calculate total using sex_female and sex_male
    total = data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_male'].index[0])][infection] + data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_female'].index[0])][infection]
    total = "{:,}".format(total)
    data_hosped['variable'] = data_hosped['variable'].str.replace('antibacterial_brit', 'total')
    
    #add comma to thousands of column count
    data_hosped[infection] = data_hosped[infection].apply(lambda x : "{:,}".format(x))
    #make new column with counts and rates
    data_hosped[infection+'_rate_count'] = data_hosped[infection].astype(str) + ' (' + data_hosped[infection+'_rate'].astype(str) + ')'
    
    #replace total (after making new column of counts and rates)
    data_hosped.at[int(data_hosped[data_hosped['variable']=='total'].index[0]),infection+'_rate_count'] = total

    #drop redundant columns and rename the needed column
    data_hosped = data_hosped.drop([infection, infection+'_rate'], axis=1)
    data_hosped = data_hosped.rename(columns={infection+'_rate_count': infection})
    #replace any row with no value (=0) with a dash 
    data_hosped[infection] = data_hosped[infection].replace({'0 (0.0)': '-'})
    #replace count and rate if rate contains SN
    data_hosped.loc[data_hosped[infection].str.contains('SN', regex=False), infection] = 'SN' 

    return data_hosped

In [12]:
summary_table_cases_lrti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_incdt_cases.csv')
summary_table_cases_lrti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_prevl_cases.csv')
summary_table_cases_urti_combined_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_incdt_cases.csv')
summary_table_cases_urti_combined_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_prevl_cases.csv')
summary_table_cases_uti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_incdt_cases.csv')
summary_table_cases_uti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_prevl_cases.csv')
summary_table_cases_sinusitis_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_incdt_cases.csv')
summary_table_cases_sinusitis_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_prevl_cases.csv')
summary_table_cases_otmedia_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_incdt_cases.csv')
summary_table_cases_otmedia_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_prevl_cases.csv')
summary_table_cases_ot_externa_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_incdt_cases.csv')
summary_table_cases_ot_externa_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_prevl_cases.csv')
summary_table_cases_urti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_incdt_cases.csv')
summary_table_cases_urti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_prevl_cases.csv')
summary_table_cases_cough_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_incdt_cases.csv')
summary_table_cases_cough_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_prevl_cases.csv')
summary_table_cases_cough_cold_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_incdt_cases.csv')
summary_table_cases_cough_cold_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_prevl_cases.csv')
summary_table_cases_throat_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_incdt_cases.csv')
summary_table_cases_throat_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_prevl_cases.csv')

In [13]:
summary_table_lrti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_incdt.csv')
summary_table_lrti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_prevl.csv')
summary_table_urti_combined_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_incdt.csv')
summary_table_urti_combined_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_prevl.csv')
summary_table_uti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_incdt.csv')
summary_table_uti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_prevl.csv')
summary_table_sinusitis_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_incdt.csv')
summary_table_sinusitis_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_prevl.csv')
summary_table_otmedia_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_incdt.csv')
summary_table_otmedia_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_prevl.csv')
summary_table_ot_externa_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_incdt.csv')
summary_table_ot_externa_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_prevl.csv')
summary_table_urti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_incdt.csv')
summary_table_urti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_prevl.csv')
summary_table_cough_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_incdt.csv')
summary_table_cough_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_prevl.csv')
summary_table_cough_cold_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_incdt.csv')
summary_table_cough_cold_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_prevl.csv')
summary_table_throat_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_incdt.csv')
summary_table_throat_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_prevl.csv')

In [14]:
#select relevant columns of each df
summary_table_lrti_incdt = summary_table_lrti_incdt[['variable', 'count']]
summary_table_lrti_prevl = summary_table_lrti_prevl[['variable', 'count']]
summary_table_urti_combined_incdt = summary_table_urti_combined_incdt[['variable', 'count']]
summary_table_urti_combined_prevl = summary_table_urti_combined_prevl[['variable', 'count']]
summary_table_uti_incdt = summary_table_uti_incdt[['variable', 'count']]
summary_table_uti_prevl = summary_table_uti_prevl[['variable', 'count']]
summary_table_sinusitis_incdt = summary_table_sinusitis_incdt[['variable', 'count']]
summary_table_sinusitis_prevl = summary_table_sinusitis_prevl[['variable', 'count']]
summary_table_otmedia_incdt = summary_table_otmedia_incdt[['variable', 'count']]
summary_table_otmedia_prevl = summary_table_otmedia_prevl[['variable', 'count']]
summary_table_ot_externa_incdt = summary_table_ot_externa_incdt[['variable', 'count']]
summary_table_ot_externa_prevl = summary_table_ot_externa_prevl[['variable', 'count']]
summary_table_urti_incdt = summary_table_urti_incdt[['variable', 'count']]
summary_table_urti_prevl = summary_table_urti_prevl[['variable', 'count']]
summary_table_cough_incdt = summary_table_cough_incdt[['variable', 'count']]
summary_table_cough_prevl = summary_table_cough_prevl[['variable', 'count']]
summary_table_cough_cold_incdt = summary_table_cough_cold_incdt[['variable', 'count']]
summary_table_cough_cold_prevl = summary_table_cough_cold_prevl[['variable', 'count']]
summary_table_throat_incdt = summary_table_throat_incdt[['variable', 'count']]
summary_table_throat_prevl = summary_table_throat_prevl[['variable', 'count']]

In [15]:
#merge baseline dfs
dfs_tab = [summary_table_lrti_incdt, summary_table_lrti_prevl, 
        summary_table_urti_combined_incdt, summary_table_urti_combined_prevl, 
        summary_table_uti_incdt, summary_table_uti_prevl, 
        summary_table_sinusitis_incdt, summary_table_sinusitis_prevl, 
        summary_table_otmedia_incdt, summary_table_otmedia_prevl, 
        summary_table_ot_externa_incdt, summary_table_ot_externa_prevl, 
        summary_table_urti_incdt, summary_table_urti_prevl, 
        summary_table_cough_incdt, summary_table_cough_prevl, 
        summary_table_cough_cold_incdt, summary_table_cough_cold_prevl, 
        summary_table_throat_incdt, summary_table_throat_prevl, 
        ]

df_final_tab = ft.reduce(lambda left, right: pd.merge(left, right, on='variable', how='outer'), dfs_tab)

df_final_tab.columns = ['variable', 'lrti_incdt', 'lrti_prevl', 
                        'urti_combined_incdt', 'urti_combined_prevl',
                        'uti_incdt', 'uti_prevl',
                        'sinusitis_incdt', 'sinusitis_prevl',
                        'otmedia_incdt', 'otmedia_prevl',
                        'ot_externa_incdt', 'ot_externa_prevl',
                        'urti_incdt', 'urti_prevl',
                        'cough_incdt', 'cough_prevl',
                        'cough_cold_incdt', 'cough_cold_prevl',
                        'throat_incdt', 'throat_prevl'
                        ]

#custom sort baseline dfs
df_final_tab['variable'] = pd.Categorical(df_final_tab['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more',
                                                                    'sex_female', 'sex_male',
                                                                    'CCI_cat_very_low', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_high', 'CCI_cat_very_high', 
                                                                    'flu_vaccine_no', 'flu_vaccine_yes',
                                                                    'bmi_cat_underweight', 'bmi_cat_healthy_weight', 'bmi_cat_overweight', 'bmi_cat_obese', 'bmi_cat_unknown',
                                                                    'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire',
                                                                    'imd_very_affluent', 'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_very_unaffluent', 'imd_unknown',
                                                                    'ethnicity_non-white', 'ethnicity_unknown', 'ethnicity_white',
                                                                    'smoking_status_ex_smoker', 'smoking_status_never_smoked', 'smoking_status_smoker', 'smoking_status_unknown',
                                                                    'season_autumn', 'season_spring', 'season_summer', 'season_winter',
                                                                    'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic',
                                                                    'antibacterial_brit'])
df_final_tab = df_final_tab.sort_values("variable").reset_index(drop=True)

In [16]:
#select relevant columns of each df
summary_table_cases_lrti_incdt = summary_table_cases_lrti_incdt[['variable', 'count']]
summary_table_cases_lrti_prevl = summary_table_cases_lrti_prevl[['variable', 'count']]
summary_table_cases_urti_combined_incdt = summary_table_cases_urti_combined_incdt[['variable', 'count']]
summary_table_cases_urti_combined_prevl = summary_table_cases_urti_combined_prevl[['variable', 'count']]
summary_table_cases_uti_incdt = summary_table_cases_uti_incdt[['variable', 'count']]
summary_table_cases_uti_prevl = summary_table_cases_uti_prevl[['variable', 'count']]
summary_table_cases_sinusitis_incdt = summary_table_cases_sinusitis_incdt[['variable', 'count']]
summary_table_cases_sinusitis_prevl = summary_table_cases_sinusitis_prevl[['variable', 'count']]
summary_table_cases_otmedia_incdt = summary_table_cases_otmedia_incdt[['variable', 'count']]
summary_table_cases_otmedia_prevl = summary_table_cases_otmedia_prevl[['variable', 'count']]
summary_table_cases_ot_externa_incdt = summary_table_cases_ot_externa_incdt[['variable', 'count']]
summary_table_cases_ot_externa_prevl = summary_table_cases_ot_externa_prevl[['variable', 'count']]
summary_table_cases_urti_incdt = summary_table_cases_urti_incdt[['variable', 'count']]
summary_table_cases_urti_prevl = summary_table_cases_urti_prevl[['variable', 'count']]
summary_table_cases_cough_incdt = summary_table_cases_cough_incdt[['variable', 'count']]
summary_table_cases_cough_prevl = summary_table_cases_cough_prevl[['variable', 'count']]
summary_table_cases_cough_cold_incdt = summary_table_cases_cough_cold_incdt[['variable', 'count']]
summary_table_cases_cough_cold_prevl = summary_table_cases_cough_cold_prevl[['variable', 'count']]
summary_table_cases_throat_incdt = summary_table_cases_throat_incdt[['variable', 'count']]
summary_table_cases_throat_prevl = summary_table_cases_throat_prevl[['variable', 'count']]

In [17]:
#merge hosped dfs
dfs_tab_cases = [summary_table_cases_lrti_incdt, summary_table_cases_lrti_prevl,  
                summary_table_cases_urti_combined_incdt, summary_table_cases_urti_combined_prevl, 
                summary_table_cases_uti_incdt, summary_table_cases_uti_prevl, 
                summary_table_cases_sinusitis_incdt, summary_table_cases_sinusitis_prevl, 
                summary_table_cases_otmedia_incdt, summary_table_cases_otmedia_prevl, 
                summary_table_cases_ot_externa_incdt, summary_table_cases_ot_externa_prevl, 
                summary_table_cases_urti_incdt, summary_table_cases_urti_prevl, 
                summary_table_cases_cough_incdt, summary_table_cases_cough_prevl, 
                summary_table_cases_cough_cold_incdt, summary_table_cases_cough_cold_prevl, 
                summary_table_cases_throat_incdt, summary_table_cases_throat_prevl, 
                ]

df_final_tab_cases = ft.reduce(lambda left, right: pd.merge(left, right, on='variable', how='outer'), dfs_tab_cases)

df_final_tab_cases.columns = ['variable', 'lrti_incdt', 'lrti_prevl', 
                                'urti_combined_incdt', 'urti_combined_prevl',
                                'uti_incdt', 'uti_prevl',
                                'sinusitis_incdt', 'sinusitis_prevl',
                                'otmedia_incdt', 'otmedia_prevl',
                                'ot_externa_incdt', 'ot_externa_prevl',
                                'urti_incdt', 'urti_prevl',
                                'cough_incdt', 'cough_prevl',
                                'cough_cold_incdt', 'cough_cold_prevl',
                                'throat_incdt', 'throat_prevl'
                                ]

#custom sort factors of variables column 
df_final_tab_cases['variable'] = pd.Categorical(df_final_tab_cases['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more',
                                                                                'sex_female', 'sex_male',
                                                                                'CCI_cat_very_low', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_high', 'CCI_cat_very_high', 
                                                                                'flu_vaccine_no', 'flu_vaccine_yes',
                                                                                'bmi_cat_underweight', 'bmi_cat_healthy_weight', 'bmi_cat_overweight', 'bmi_cat_obese', 'bmi_cat_unknown',
                                                                                'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire',
                                                                                'imd_very_affluent', 'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_very_unaffluent', 'imd_unknown',
                                                                                'ethnicity_non-white', 'ethnicity_unknown', 'ethnicity_white',
                                                                                'smoking_status_ex_smoker', 'smoking_status_never_smoked', 'smoking_status_smoker', 'smoking_status_unknown',
                                                                                'season_autumn', 'season_spring', 'season_summer', 'season_winter',
                                                                                'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic', 
                                                                                'antibacterial_brit'])
df_final_tab_cases = df_final_tab_cases.sort_values("variable").reset_index(drop=True)

## 1- process data to make a table of counts and rates, and save it

In [19]:
data_hosped = data_process(df_final_tab, df_final_tab_cases, 'lrti_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'lrti_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'lrti_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'lrti_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_abs_prevl')

#save hosped count and rate table
data_hosped.to_csv(f'../output/aggregate_cases_table/cases_count_rate_table_all_infections.csv', index=False)